# Key Methods in Physical Geography
# Computing Practical 4: Spatial Analysis of Geographic Data

This is a Jupyter notebook running on the University of Edinburgh Noteable Service (https://www.ed.ac.uk/information-services/learning-technology/noteable/about). Notebooks are interactive documents that can contain computer code (Python for this course), text and images.

For some components in this notebook, data files will be used. Any data file that is used for plotting with this and other notebooks needs to be uploaded to the Jupyter server -- unless they are provided for you, or there are commands provided to download the files directly to the Jupyter practical folder.

### Key Learning Outcomes

- Reading and Plotting Raster Data: GeoTIFF files

- Comparing point (vector) and raster data

    - Overlaying point (vector) data on raster data
    - Interpolating from raster to a point location
    - Interpolating multiple points through batch processing
    - Analysing comparison 
        - Scatter plots
        - Descriptive statistics
        - Linear regression 
    - Saving output as .csv files

- Raster generation from irregularly spaced data

- (Hill)slope analysis

    - Finding gradients
    - Logical slicing

- Reading and plotting climate data: NETCDF files

## Reading and plotting raster data

Last practical we learned how to read into python, extract data from, and analyse a very common type of GIS data format, shapefiles -- using the python library `geopandas`. Shapefiles describe *vector* data. Today we will see how to analyse another very common type of data format, *GeoTIFF* files (\*.tif or \*.tiff) files. (Tag Image File Format files, or TIFF files, are often used for images; GeoTIFF files have georeferencing encoded.) Geotiff files are a common file format used to store *raster* data, i.e. gridded data. 

(There is another type of file format, used more often to store climate data but equally versatile, called *netcdf* (Network Common Data Form) -- there is a brief exercise with netcdf files at the end of the practical.)

We will work with a GeoTIFF file which contains a Digital Terrain Model (DTM) of Arthur's Seat at 2m horizontal resolution. The file is called `arthurs_seat_small.tif` and it has been included in your directory.

Begin by importing the library `rasterio`, but given the abbreviation `rio` as some suggested code below might depend on this abbreviation.

We begin by using the function `rio.open` to open the geotiff file. This returns a variable (or type `DatasetReader`) which can be used to extract data from the geotiff. In terms of last week's practical, this DatasetReader is analogous to the `geodataframe` we encountered next week. The process is very similar: you will define a new variable (e.g. `geotiffreader`) and set this equal to `rio.open([filename])` where `[filename]` is the geotiff you wish to analyse. 

In the following cell, use `rio.open()` to set a new `DatasetReader` to extract data from `arthurs_seat_small.tif`.

(Not as many commands will be given to you for copy and paste as in previous practicals -- this is to prepare you for your assessment. If it is not clear how to use the `open` function, please refer to example in last week's practical under the heading "**Overlying on rasters**"
which opens the file `dem.tif`. The same applied for the following task as well.)

If `rio.open()` was called correctly you can now examine the properties of the geotiff. One obvious question is, what does the data look like? We can address this by extracting the data as a 2 Dimensional array. The `geotiffreader` variable (assuming you called it this) has a bespoke function `read` which can be used for this purpose. In the following cell, define a variable `z` to be equal to `geotiffreader.read(1)`. The argument `1` tells python to read only the *first layer* (even though the file contains only one layer). Without it, `z` would be a 3 Dimensional array and require more work to visualise.

Now we can visualise this DTM with the function `imshow`, which is part of the `matplotlib.pytplot` module. Import `matplotlib.pytplot` as `plt` and then type `plt.imshow(z)` in the next cell. 

In [ ]:
import matplotlib.pyplot as plt

You should see a color image of a top down view of Arthur's seat. However, there is not much information given. 

*In the following exercises you will make changes to the figure above. You might wish to modify the cell above, or create new cells below this one by clicking the "Insert cell" button on the Notebook menu bar -- just below "File"*.

<span style="color:blue">**Adding a colorbar.**</span> 

Firstly, there is no colorbar. This can be added with the command

```python
cbar = plt.colorbar()
```

`cbar` is a "handle" to the colorbar which can be used to modify it further (and can have a different name). Modify the figure above to have a colorbar and run it again.

You can further modify the colorbar using `cbar`. For instance `cbar.set_label()` allows you to label the colorbar by providing a string as an argument. `cbar.set_ticks()` can change the tick locations on the colorbar and takes a *list* of locations as an argument. Modify the code further to add the colorbar label <span style="color:red">'Elevation (m)'</span> and only have ticks at 125 and 250 m.

<span style="color:blue">**Changing the colour map.**</span> 

The colour map (but python spells it **color**!) determines which color corresponds the value in each pixel. By default the color map is `'viridis'` but you can change this as `imshow` has an **optional argument** `cmap`. Recall: when a function has an **optional argument**, we must reference this argument *by name* (e.g. `cmap=...`) within the brackets of the function call. In the figure above, can you change the color map to the rainbow scale `'jet'`? (Note, the name of the color map must be in quotes.) Experiment with a few color maps listed [here](https://matplotlib.org/tutorials/colors/colormaps.html) (you need to scroll down to see the choices). How does Arthurs Seat appear with a seismic color map? (**Tip**: when dealing with data that can be positive or negative, diverging colormaps are good to use -- but only if configured if their "diverging" point coincides with zero!)

<span style="color:blue">**Changing axis georeferencing.**</span> 

The x- and y-axis ticks are not in terms of physical distance, but rather in terms of *pixel count*. (e.g. where it says "1200" on the x-axis, this is the position of the 1200th column of the raster.) This is not helpful for comparing with features on other maps, or with other measurements of elevation. 

This is addressed in `imshow` with the *optional argument* `extent`. This is used (within the brackets of the `imshow` command) as 

```python
extent = (left, right, bottom, top)
```

where `left` is the leftmost extent of the raster in its geographic coordinate system, `right` is the rightmost extent, etc. **But how do we know the geographic extent of the raster file?** In e.g. QGIS these are properties we can look up in a table when we load a GeoTIFF. In python we can do something similar. Our variable `geotiffreader` which we used to extract the array also contains these bounds in the property `geotiffreader.bounds`. In the cell below, type `print(geotiffreader.bounds)` to see what these are.

Use the `extent` optional argument to have the x-labels show proper geographic positioning. You can modify the axes to show the correct georeferencing in one of two ways: (1) add the optional `extent` argument as described above by typing the numbers you see below in place of `left`, `right`, etc. or (2) previous to the `imshow` command, define the variables

```python
figTop = geotiffreader.bounds.top
figBottom = geotiffreader.bounds.bottom
```

and so on, and then adding the optional `extent` argument with these variables in place of the numbers below. Methods (2) is better practice -- as this code can be reused for a different GeoTIFF with different extents.

You may notice that the y-axis values increase downward; this is an ideosyncracy of GeoTIFF referencing -- the nothernmost (topmost) corner is treated as the "bottom". To correct this, "top" and "bottom" need to be reversed in `extent`.

<span style="color:blue">**Labelling axes.**</span> 

Labelling of the x- and y-axis works exactly has it has before. Label the axes "x (m)" and "y (m)" . *How do we know they are in meters? See below*

<span style="color:blue">**Geographic coordinate reference system**.</span> 

No more changes to the current figure are required (though we have lots more to do!) but we should be aware of the **coordinate reference system** used in the GeoTIFF. This is a concept that you should have learned about in the context of GIS -- different data can be referenced to different *grids* -- latitude/longitude, Polar Stereographic, Universal Transverse Mercator (UTM), etc. When we want to compare data in different coordinate systems, we need this information. For instance, data posted in UTM can easily be transformed to lat-lon coordinates using a number of python functions.

We can find the coordinate reference system (CRS) by examining `geotiffreader.crs` -- if we `print(geotiffreader.crs)` we see that the CRS is EPSG.27700. We would need to look this up but it is the British National Grid (that used by the Ordnance Survey). Similarly `geotiffreader.crs.linear_units` tells us that the bounds given are in metres.

## Comparing point (vector) and raster data

Some of your fellow classmates trekked through Holyrood park with GPS devices to measure location and elevation. To make full disclosure, the data you are about to use is **not** this data -- this is because of a mixup either with the GPS device settings or the georeferencing data of the GeoTIFF plotted above, but regardless this data would not have made for an interesting exercise. The data you use is synthetic randomly generated data, meant to simulate measurements taken in the locations where data was gathered, with similar levels of error.

There is a .csv file in your practical folder titled `arthurs_gps.csv` with data columns $X$, $Y$, and $elev$. In the following cell, using similar techniques to those learned in week 7, read the data from the file into a pandas dataframe `gps_data`. Just as before, you can then type `gps_data` to see the contents, or you can also type `print(gps_data.keys())` to see just the header names. 

### Overlaying point (vector) data on raster data

The easiest way to visually compare the GPS and raster data is by overlaying a plot of the point data on the raster. In the following cell, create a raster plot of elevation as you did above. 

Immediately below create a **scatter plot** of the gps data. This is done with the function `plt.scatter()`. `plt.scatter` is called just as with `plt.plot`, with the `x` and `y` values as the first two arguments -- these will be the $X$ and $Y$ columns fo the .csv file (i.e. `gps_data.X` etc). The *elevation* data from the .csv file is used through the `scatter` optional argument `c`. Set `c` equal to `gps_data.elev`. 

For the scatter plot, we would also like clear markers. To more clearly distinguish the markers, use the optional argument `edgecolor` (i.e. set it equal to `'k'`). Make sure as well that the optional `cmap` argument is the same colormap as for `imshow`. 

If done correctly, you should see a raster plot of elevation, with small filled circles whose color indicates the value from the GPS measurement, not the raster. From this you can visually gauge the level of agreement. (You can also change the markers from circles to e.g. squares if you like -- see [the documentation](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.scatter.html).)

However, we can make a more quantitative comparison. We are going to use the GPS data to see how to interpolate raster data to specific point locations. Such operations can be extremely usefule in geography dissertations, e.g. interpolating climate data to a specific location for comparison with a weather station. 

<span style="color:red">Unfortunately `rasterio` does not include a good function for interpolation, so we will need to do it a bit more.. manually -- that is, by using more basic Python functionality. The extra work we do, though, will help us with another task below. At the moment, we are interpolating from a gridded product to irregularly spaced points. Later on we will interpolate from irregularly spaced data to a grid -- a far more powerful tool (as data is generally gathered irregularly.)</span>

### Interpolating from raster to a point location

In order to interpolate DTM values to the locations in the .csv file, we need additional structures: NumPy arrays describing the coordinates of the raster grid. With a netcdf file these structures would be provided -- but for a GeoTIFF we need to create them ourselves.

That is, we need an array that contains the $y$-positions of all *rows* in the raster, and an array that describes the $x$-positions of all *columns* in the raster. For this we can use the `arange` function. In the cell below, `Y_array`, the array describing the positions of all *rows* is generated. There are several things to note about this: 

(1) We use the bounds of the raster file as the lower and upper bounds of this array.

(2) Since "top" is actually *below* "bottom" due to GeoTIFF convention, we start with "bottom" and end with "top" -- but the step is equal to **-2**, not 2, even though there is a 2m spacing.

**This consideration must only be made for GeoTIFF data and not for other types of data, such as NetCDF**.

(3) The beginning of the range in `np.arange` is offset by 1 meter. Remember that the raster is composed of 2m pixels. We also know (though it is buried deep in the metadata) that the data locations are in the *centre* of the 2m x 2m pixel. The "top" position we use is actually the *bottom* (i know, very confusing) of the lowest pixel. The centre of the pixel is offset 1 metre from this.

Add code to define `X_array`, the positions of columns, similarly. The offset for the lower bound should be +1, rather than -1.

In [ ]:
# this is imported because we have not yet done so in this notebook!!!
import numpy as np

# arange starts with top+1 as the data locations are actually at the center of each 2m pixel.
Y_array = np.arange(geotiffreader.bounds.bottom-1,geotiffreader.bounds.top,-2);

# your code to generate X_array -- the array of column positions


Now we are ready to interpolate. To begin, import `interp2d` from the module `scipy.interpolate` in the cell below -- it is the function we will use.

Interpolation is still unfortunately a two-step process, illustrated in the cell below. In the first step, we use the position arrays `X_array` and `Y_array` together with the raster data elevation array `z` to create an **interpolant** -- a new *function* which can then produce interpolated values of elevation for *any* set of coordinates. The line of code looks like this:

```python
interper = interp2d(X_array,Y_array,z)
```

In the above, `interp2d` is called with 3 arguments -- the grid coordinate arrays `X_array` and `Y_array`, and the raster array `z`. There are additional optional arguments describing the *type* of interpolation (`'linear'`,`'cubic'`, or `'quintic'`). We will use the default, `'linear'`. 

`interper` is then a variable which is set equal to the result of `interp2d`. Here is where it becomes confusing -- this *variable* is a **function**!!. It is a bespoke function -- with a name we choose ourselves -- which can then be used to find the interpolated values of any coordinates we query. The function is structured this way so that each time we have new coordinates to interpolate, the results are found by Python a little bit faster.

In the second step below, we provide our **first** GPS point position to `interper` and assign the result to a new variable, `interp_elev0`. The interpolated and measured values are then printed.

Make sure you understand this code. You and try and modify it to interpolate e.g. the 20th position in the .csv file, and compare this against the 20th measurement.

In [ ]:
interper = interp2d(X_array,Y_array,z)

# what are the arguments for interper?
interp_elev0 = interper(gps_data.X[0],gps_data.Y[0])

print('interpolated value: ' + str(np.round(interp_elev0,1)))
print('measured value: ' + str(np.round(gps_data.elev[0],1)))      

### Batch processing for entire excel file.

Hopefully you see the problem -- we don't want to do this 49 times!!! This is a job for **batch processing**. We are going to use a for-loop!

In order to end up with an array of interpolated values the same size as the columns in the .csv file, there is a new technique required -- creating an *empty* array. We have seen how to create arrays with `np.arange` but this is for a bespoke purpose -- we might want an array to be a placeholder for values not yet calculated. This is done with `np.empty`. It means the array is created, but the contents are meaningless (the values could be zero, or a billion -- the point is they are meaningless until we assign them).

For instance, to create an empty array `interp_elev` of size 49, the number of gps points, we type:

```python 
interp_elev = np.empty(49)
```

Then, to insert a value into the `i`th position, we type 

```python 
interp_elev[i] = [value]
```

In the following cell, `interp_elev` is created with `np.empty`. Next, the skeleton of a `for`-loop is started but is unfinished. Essentially, for each `i`, we want to interpolate elevation using `interper` to the `i`th GPS location. In the cell above, this is done for the *zeroth* position only. Finish the for-loop below with a modified call to `interper`. 


In [ ]:
interp_elev = np.empty(49)

for i in range(49):
    
    # update the i-th value of interp_elev with the interpolated value at the ith X and Y positions.
    
    

### Scatter plot comparison

If done correctly, `interp_elev` should contain interpolated values corresponding to the GPS coordinates, and these can be compared against the measured values in a scatter plot. In the cell below, create a scatter plot (using `plt.scatter`) with `interp_elev` on the x-axis and `gps_data.elev` on the y-axis. (No need to format the figure.. unless you want to!)

What would the figure look like if there were was perfect agreement in the measurements? To help guide an assessment of measurement error, plot a straight line in the figure as well. This can be done by adding

```python
plt.plot(interp_elev,interp_elev)
```

in the same cell. Given an eyeball estimate of how accurate the measurements are.

### Descriptive statistics of error

We can be a bit more accurate in terms of assessing measurement accuracy. Firstly, we can define the error with a new valuable, `gps_error = gps_data.elev - interp_elev`. In the following cell, define `gps_error` and find its mean and standard deviation.

Though this data is a bit conjured, it is still important to remember that the error due to measuring elevation with GPS is due both to the error in one's horizontal position (which is small) as well as the error in measuring elevation (which is large).

### Linear regression

We can also use linear regression to fit a **line** to this scatter plot, describing the gps-measured elevation as a **function** of raster elevation:

\begin{equation}
\mathrm{GPS\ Elevation} = M \times \mathrm{Raster\ Elevation} + C
\end{equation}

In the language of linear equations, $M$ is the *slope* of the line, and $C$ is the intercept. 

We can do this with the function `scipy.stats.linregress`. This function is called as follows:

```python
m,c,r,p,_ = linregress(x,y)
```

where `x` is what is commonly known as the *independent variable*, i.e the variable that explains `y`, the *dependent variable*. In the present case we are treating the ground GPS measurements as the dependent variable (although in many applications the situation is reversed -- with the ground measurements treated as the independent variable and the far less certain remotely sensed measurements treated as the dependent variable). `m` and `c` correspond to the slope and intercept, while `r` is the *correlation* between the two data sets, and `p` is the *significance*. A correlation is considered *strong* if it is close to 1 (or -1), and is considered *significant* if `p` is less than 0.05. (For more details, see the file correlation.pdf in this week's Learn folder.)

In the follow cell, import `linregress` from `scipy.stats` and find the linear regression between `interp_elev` and `gps_data.elev`. What are the slope and the intercept? What is the correlation and its significance (p-) value?

In [ ]:
from scipy.stats import linregress

### Saving arrays as .csv files

A useful technique is to be able to save the contents of arrays to a text or .csv file, to allow you (if you want to) to analyse in Excel or some other application. There are ways to do this with `pandas` but the fastest option is `np.savetxt()`. The function is called as

```python
np.savetxt(filename,array)
```

Here `filename` is a **string** indicating the name of the file -- and you can give it a .csv extension (though the extension will not affect how the file is saved). Thus, `filename` should have quotes! And `array` is the data you are saving to a file, for instance `interp_elev`. In the following cell, save the `interp_elev` array to the file `'interp.csv'`.

If done correctly, `'interp.csv'` will appear in your practical folder. Though noteable will not allow to you to *download* it (recall my announcement about this -- you can always move it to another location and download), you can easily view it by clicking on it. You will see that the values are in a very long, difficult to read format (scientific notation). You can change this to decimal format by modifying the call to `savetxt` with an *optional argument* `fmt`. If `fmt` is set equal to `'%10.5f'` for instance (do not forget quotes!) then python will save values in decimal format with 5 spaces. After rerunning, click on the file (or refresh the page where it is displayed) to view this.

If you have multiple columns and you would like to create a .csv, then it is also good practice to use the optional argument `delimiter` and set it to `','`. By default the delimiter is a space -- i.e. in the text file, columns will be separated by spaces.

`np.savetxt()` does not allow column headers to be written -- but it is likely easier to write these in manually in excel then to configure Python to do this.

## Raster generation from irregularly spaced measurements

Here we have started with a gridded DTM (`arthurs_seat_small.tiff`). But in fact, these gridded products are derived from irregularly spaced data. Let us pretend that we *only* have our GPS data and try to derive a gridded raster product.

We can do this with the function `scipy.interpolate.griddata`. `griddata` is similar to `interp2d` in that it interpolates data -- but rather than interpolating from a gridded data set to a point, it interpolates from *irregularly spaced points* to a grid.

`griddata` is called as:

```python
gridded_elev = griddata((x_points,y_points),point_data,(X,Y))
```

where `x_points` and `y_points` are the coordinates of the irregularly spaced data, with `point_data` the corresponding values. 

`X` and `Y` are coordinates describing a grid -- where `X` is the position of the columns, and `Y` is of the rows. The arrays you created for interpolation, `X_array` and `Y_array`, are *almost* suitable for this, but `griddata` expects 2 dimensional arrays describing the grid to which we interpolate. This is addressed by `numpy.meshgrid`:

```python
X_array_2d, Y_array_2d = np.meshgrid(X_array,Y_array)
```
takes as input the 1-D arrays `X_array` and `Y_array` describing column and row positions, and returns 2D arrays `X_array_2d`  and `Y_array_2d`, which are now gridded values of $x$ and $y$ positions, respectively -- for every pixel location in the elevation raster, `X_array_2d` and `Y_array_2d` describe its position.

In the following cell, produce a gridded data product from the .csv data using `griddata`, by creating 2D arrays of $x$ and $y$ coordinates with `meshgrid`, and then calling `griddata`. In the example of `griddata` above, the variables are given different names than the ones you will likely use in your function call. Can you figure out which variables to use where?

In [ ]:
from scipy.interpolate import griddata

if done correctly, `gridded_elev` is now a 2D array that can be plotted as a raster, just like `z`. To compare them side by side, we need to learn how to create **subplots**. To create subplots, we begin with the call 

```python
plt.subplots()
```

Let's have 2 figures side by side. This means the next command should be

```python
plt.subplot(121)
```

This tells matplotlib that we are going to have 1 **row** of columns (the first "1"), and two **columns** ("2"). The last "1" indicates that we are about to draw the first figure.

If, immediately below this, we place code to draw a figure, it will correspond to the first figure. Plot (again) the elevation raster `z` with `plt.imshow()` as you did before. (colormap is up to you.)

Now, in the same cell, add the second subfigure, by calling `plt.subplot(122)`, where the second "2" indicates the second subplot. Here, create a near-identical figure, but plot `gridded_elev`, the array created with `griddata`, rather than `z`.

The figures are likely smaller than you would like -- and the colorbars too large. 

We can make the plots larger on screen by passing the optional argument `figsize` to `subplots()` e.g. `plt.subplots(figsize=(5,10))`. We can also resize the colorbars by passing the optional argument `shrink` to `colorbar()` e.g. `colorbar(shrink=.2)`. (The smaller the value, the smaller the colorbar.) Experiment with `figsize` and then `shrink` to get an image that you like. 

Now compare the GeoTIFF raster to the one we created by (synthetic) point measurements. There is a lot of empty grid -- mainly because the measurements did not cover a wide enough area. (In the future if the whole cohort is involved, we can map out the entire park!)

Where there are values, how accurate does it appear? There is an additional optional argument for `griddata` called method -- for **method of interpolation**. There are three methods: `'linear'`,`'cubic'`, and `'nearest'` (nearest neighbour). How does each affect the figure on the right above?

## Hillslope Analysis

In many cases analysis of a spatial data set involves determining how quickly it is changing as one moves across it i.e. its *variability*. One measure of how much terrain varies is the **slope** of a DTM. With our NumPy tools this is something we can investigate easily by finding its **gradient**:

```python
gradeEast, gradeNorth = np.gradient(z, dx)
```

here `dx` is the spacing of the raster pixels -- in the present case, 2. `gradeEast` is the *grade*, or *slope* in terms of rise over run, in the Eastward direction at each pixel, and similarly for gradeNorth.

However, the direction in which the slope is steepest may not be either east or north! We find this "steepest slope" at each pixel with the formula

\begin{equation}
\mathrm{grade} = \sqrt{gradeEast^2 + gradeNorth^2}
\end{equation}

However, spatial analysts are less concerned with "rise over run" and more with slope *angle*, the angle of ascent (or descent) of a slope. Once you have calculated *grade*, try to calculate the slope angle. You might recall `np.arctan()` which will be useful! However, `np.arctan()` gives results in radians, and we want degrees -- apply `np.rad2deg` to the result to get a 2 Dimensional array of slope angle in degrees.

Use `imshow` to visualise this slope angle. A lot of detail is missing though because of some very steep isolated regions (angles of up to 80 degrees). We can change the limits of the raster plot by passing two additional optional arguments to `imshow`: `vmin` and `vmax`, the lower and upper bounds. Try plotting with bounds of 0 and 45 degrees -- can you see more detail in the relatively flat areas? (Try an upper bound of 30 degrees as well -- this is too low. Much of the figure becomes *saturated*. This is the sort of figure to avoid in reports!)

### Logical slicing with raster data

Is Arthur's Seat steeper at higher elevations? Is there an easy way to assess this?

The easiest way that *I* can think of is through the use of *logical slicing* -- which can be applied to both 1 and 2 dimensional arrays, and will still return a list of values that can be used for statistics.

We will generate a list of slope angle values above a certain elevation (100m) and another list of slope values below this elevation, and plot histograms of each.

In the next cell create a new variable `highangle` via

```python
highangle = slopeAngleDegrees[z >= 100]
```

(NB. This assumes that your array of slope angle is called `slopAngleDegrees`, though it might not be as I didn't suggest a name!)

`highangle` is a 1D array composed of the slope angles in all the pixels where elevation is 100m or higher. It does not matter that `z` is 2 Dimensional, logical slicing works as in 1D. Create a similarly defined array `lowangle` of slopes below 100m as well. Next, create a histogram for `lowangle` and `highangle`. (No need to do any formatting, just use the python defaults.) In between the histograms, if created in the same cell, you will need to have a call to 

```python
plt.figure()
```

in order to create a new figure. Just from visual assessment, do you think Arthur's Seat is steeper at higher or lower elevations?

## Reading and plotting climate data: NETCDF files

GeoTIFFs are only one type of data format. A common format for climate data is netcdf (Network Common Data Form). Netcdf files can accomodate more complex data structures than GeoTIFF -- and importantly, they are often **not readable with GIS software such as QGIS or ArcGIS**. This is not universally true, but many dissertation students struggle with netcdf files as they are not able to view or analyse them with GIS software.

In your practical folder is a file called `sresa1b_ncar_ccsm3-example.nc` (the extension for netcdf files in ".nc"), downloaded [here](https://www.unidata.ucar.edu/software/netcdf/examples/files.html). The file contains a few output data fields from one timestep (on 16 May 2000) from a simulation with the Community Climate System Model (CCSM). We will extract one of these data fields and visualise it, in the process learning how to work with netcdf files.

### The xarray module

There are a number of python modules that can be used to read netcdf files; we will use `xarray`.

In the cell below, import `xarray` as `xr`.

We "open" a netcdf file using `xr.open_dataset()`, which takes the file name as an argument. This returns a `dataset` variable, which can be used to extract data:
```python
ds = xr.open_dataset('sresa1b_ncar_ccsm3-example.nc')
```
After this type `ds`, and this allows a view of the contents of the data file.

We see a full list of coordinates and data variables. At the top of the view, we see that there are 128 latitude levels and 256 longitude levels. There are also a number of data variables, such as `pr` (precipitation), which is a function of time, latitude and longitude; and `ua` (wind speed), a function of time, latitude, longitude, and pressure level -- atmospheric models use pressure, not elevation, as a vertical coordinate. (It would not be possible to package this diverse data into a GeoTIFF file, and it would be difficult to extract the data with QGIS.)

Data is extracted as follows. For latitude, we write
```python
latitude = ds['lat'].values
```
which saves the latitude coordinates into the array `latitude`. Other variables are extracted similarly -- with the string in square brackets indexing the correct variable.

In the following cell, extract latitude, longitude, and precipitation into arrays. 

To visualise global precipitation, we could use `plt.imshow()`, but as the netcdf file contains georeferencing arrays, it is easier to use a different plotting function, `plt.contourf()`, i.e. a filled contour plot. The `contourf()` function is called as

```python
plt.contourf(longitude,latitude,data)
```

where `data` is the field we want to visualise, e.g. `precip`. However, `contourf` requires a 2D array, and `precip` is technically a 3D array, as it depends on time, even though there is only one time level. (You can examone this by typing `print(np.shape(precip))` in the cell above.) When an array contains a "singleton" dimension, this dimension can be eliminated with the function `np.squeeze()`. When `precip` is passed as an argument to `squeeze`, the result is a 2D array. 

In the following cell, using `contourf()` to plot precipitation, using `np.squeeze()` to reduce it to a 2D array.

The figure above is global precipitation in latitude and longitude. Though we cannot see continents, we can see clear evidence of the Asian Monsoon at ~80 Longitude, where the band on strong precipitation extends farther north than over the Pacific. All the same techniquest discussed above can be used for formatting (e.g. labelling the axes, changing the colormap, adding a colorbar).

`contourf()` is distinct from `imshow()` because we can change the level of detail shown. We can specify the *number* of contours:

```python
plt.contourf(longitude,latitude,data,nlevels)
```

where `nlevels` is the number of filled contours in the image. If `nlevels` is left off Python chooses for us. Experiment with changing the number of contours -- how does the figure appear with only 4? With 100? (With 100 levels we see more detail -- the drawback being that increasing the number of contours can sometimes cause the figure to be slow to plot.) An alternative is to pass an *array* of contour levels, e.g.

```python
plt.contourf(longitude,latitude,data,np.arange(0,.0002,.00002))
```
Neither optional argument need be referenced by name -- Python understands their meaning simply there placement in the argument list.